In [0]:
import json
import os

def get_pipeline_config():
    """Reads the JSON config file from the repo."""
    # This gets the directory where THIS notebook (common_utils) is located
    current_dir = os.getcwd() 
    
    # We go up one level from 'utils' to 'Demo', then into 'config'
    config_path = os.path.abspath(os.path.join(current_dir, "..", "config", "env_config.json"))
    
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Config file not found at {config_path}. Check your folder structure!")
    
    with open(config_path, "r") as f:
        return json.load(f)

def authenticate_adls(storage_account, scope):
    """Sets configurations for direct ADLS access using Hadoop Spark Context."""
    account_url = f"{storage_account}.dfs.core.windows.net"
    
    # Get the underlying Hadoop Configuration
    # This is more robust than spark.conf.set in Shared/UC clusters
    hconf = spark._jsc.hadoopConfiguration()
    
    # Set the Auth Type
    hconf.set(f"fs.azure.account.auth.type.{account_url}", "OAuth")
    hconf.set(f"fs.azure.account.oauth.provider.type.{account_url}", 
              "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
    
    # Set the Credentials from Secrets
    hconf.set(f"fs.azure.account.oauth2.client.id.{account_url}", dbutils.secrets.get(scope, "client-id"))
    hconf.set(f"fs.azure.account.oauth2.client.secret.{account_url}", dbutils.secrets.get(scope, "client-secret"))
    hconf.set(f"fs.azure.account.oauth2.client.endpoint.{account_url}", 
              f"https://login.microsoftonline.com/{dbutils.secrets.get(scope, 'tenant-id')}/oauth2/token")
    
    print(f"✅ Hadoop Auth configurations set for {storage_account}")

def get_path(container, storage_account, relative_path):
    """Constructs a full ABFSS path."""
    # Ensure no double slashes if relative_path starts with one
    relative_path = relative_path.lstrip('/')
    return f"abfss://{container}@{storage_account}.dfs.core.windows.net/{relative_path}"